In [ ]:
### imports

# external modules
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
import LandauClassifier
importlib.reload(LandauClassifier)

In [ ]:
### load test histograms
    
histname = 'chargeInner_PXLayer_2'
dffile = '../data/DF2017_{}.csv'.format(histname)
histdf = csvu.read_csv(dffile)
histdf = dfu.select_dcson(histdf)
histdf = dfu.select_highstat(histdf)
print(len(histdf))

(hists_train) = hu.preparedatafromdf(histdf, donormalize=True, doplot=False)
print(hists_train.shape)

In [ ]:
### define a LandauClassifier

importlib.reload(LandauClassifier)
classifier = LandauClassifier.LandauClassifier( dogauss=True )

In [ ]:
### do some example reconstructions

inds = range(50896,50912)

hists = np.zeros((len(inds),hists_train.shape[1]))
recos = np.zeros((len(inds),hists_train.shape[1]))

starttime = time.time()
for i, ind in enumerate(inds):
    hists[i,:] = hists_train[ind,:]
    recos[i,:] = classifier.reconstruct(hists[i:i+1])
endtime = time.time()
print('reconstruction finished in {} seconds'.format(endtime-starttime))

for i in range(len(hists)):
    hist = hists[i]
    reco = recos[i]
    
    # with initial parameters
    landaumax = np.argmax(hist)
    landauwidth = landaumax/6.
    norm = np.sum(hist)/4.
    gausswidth = landauwidth
    xdata = np.arange(0, len(hist))
    landau = classifier.fitfunc( xdata, landaumax, landauwidth, norm, gausswidth )
    pu.plot_hists( [hist,landau], 
                    colorlist=['black','red'], 
                    labellist=['orig','landau'] )
    
    # do fit
    pu.plot_hists( [hist,reco], 
                    colorlist=['black','red'], 
                    labellist=['orig','reco'] )

In [ ]:
xdata = np.arange(0, 100)
landaumax = 20
landauwidth = 5
norm = 1
y = np.divide(xdata-landaumax,landauwidth)
landau = norm/np.sqrt(2.*np.pi)*np.exp( -(y+np.exp(-y))/2. )
plt.figure()
plt.plot(xdata, landau)

gausswidth = 0.1
xgdata = np.linspace(-5*gausswidth,5*gausswidth,num=100)
gauss = np.exp( -0.5*np.power(xgdata/gausswidth,2) )/(np.sqrt(2*np.pi)*gausswidth)
plt.figure()
plt.plot(xgdata, gauss)

conv = np.convolve(landau, gauss, mode='same')
plt.figure()
plt.plot(xdata, conv)